In [1]:
from pipeline.dl_models import MLP, CombinedMLP
from pipeline.preprocessing import feature_mfcc, feature_bandpower_struct, remove_high_frequencies
from pipeline.dataloader import PhonocardiogramAudioDataset, PhonocardiogramByIDDatasetOnlyResult
from pipeline.utils import compose_feature_label, audio_random_windowing

from tqdm import tqdm
import numpy as np

from pathlib import Path
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import re, random

In [2]:
file = Path(".") / "assets" / "the-circor-digiscope-phonocardiogram-dataset-1.0.3"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Training on actual patient audio files

def augmentation(data :np.ndarray, sr : int=4000, window_length_hz :int =200, window_len_sec :float=5.) ->np.ndarray:
    x = data
    # x = energy_band_augmentation_random_win(x, sr=sr, window_hz_length=window_length_hz)
    # x = np.fft.ifft(x).real
    x = audio_random_windowing(x, window_len_sec)
    return x

In [4]:
lookup = PhonocardiogramByIDDatasetOnlyResult(str(file / "training_data.csv"))

# Feature functions
features_fn = [
    feature_mfcc, 
    # feature_chromagram, 
    # feature_melspectrogram,
    # feature_bandpower_struct(4000,200,0.7),
    # NMF, # found -> takes around 0.1s per file
    ]    

In [5]:
def dset_trans(f : str): # each takes ~0.1s

    result = compose_feature_label(
        f,
        lookup, 
        features_fn,
        lambda ary_data : remove_high_frequencies(augmentation(ary_data,4000,200,3.), sample_rate=4000,cutoff_frequency=450).real,
        dim=2,
        is_np=False
    )

    return result

In [6]:
print([f.__qualname__ for f in features_fn])
dset = PhonocardiogramAudioDataset(
    file / "clear_training_data",
    ".wav",
    "*", # Everything
    transform=dset_trans,
    balancing=True,
    csvfile=str(file / "training_data.csv"),
    shuffle=True,
)

['feature_mfcc']


In [7]:
def split_features_labels(dataset):
    features = []
    labels = []
    for feature, label in dataset:
        features.append(feature[0])
        labels.append(label)
    return np.array(features), np.array(labels)

features, labels = split_features_labels(dset)


In [8]:
features.shape

(3058, 42)

In [9]:
labels.shape

(3058,)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("X_train.shape", X_train.shape)
print('y_train.shape', y_train.shape)

X_train.shape (2446, 42)
y_train.shape (2446,)


## Model1

In [11]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam

# Define the model
model = Sequential()

# Add convolutional layer
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))

# Add max pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.005)  # Specify learning rate within Adam optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'recall'])

# Reshape X_train for 1D CNN
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Train the model
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32)

model.summary()


Epoch 1/50


/Users/ziaoyou/WorkSpace/CodeSpace/Projects/HeartDiseasePrediction/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 991us/step - accuracy: 0.5090 - loss: 2.5883 - recall: 0.5085 
Epoch 2/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 965us/step - accuracy: 0.5393 - loss: 0.7532 - recall: 0.5234
Epoch 3/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.4988 - loss: 0.8720 - recall: 0.5214
Epoch 4/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step - accuracy: 0.5118 - loss: 0.8127 - recall: 0.5550
Epoch 5/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5402 - loss: 0.7547 - recall: 0.4342
Epoch 6/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5787 - loss: 0.6749 - recall: 0.5579
Epoch 7/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - accuracy: 0.5361 - loss: 0.6888 - recall: 0.6144
Epoch 8/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step - accuracy: 0.5727 - loss: 0.6872 - recall: 0.5239
Epoch 9/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.5476 - loss: 0.6870 - recall: 0.5779
Epoch 10/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5792 - loss: 0.6723 - 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 40, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 20, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 123,653 (483.02 KB)

 Trainable params: 41,217 (161.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 82,436 (322.02 KB)

In [12]:
# Reshape X_test for evaluation
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Evaluate the model
evaluation_results = model.evaluate(X_test_reshaped, y_test)

# Extracting individual evaluation metrics
loss = evaluation_results[0]
accuracy = evaluation_results[1]

# Printing evaluation results
print("Evaluation Results:")
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step - accuracy: 0.6275 - loss: 0.7049 - recall: 0.6120
Evaluation Results:
Test Loss: 0.6928813457489014
Test Accuracy: 0.6029411554336548


## Model2

In [37]:
from keras.layers import Dropout

# Define the model
model = Sequential()

# Add convolutional layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)  # Adjusted learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'recall'])


X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
# Train the model
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=64, validation_split=0.2)

model.summary()


Epoch 1/50


/Users/ziaoyou/WorkSpace/CodeSpace/Projects/HeartDiseasePrediction/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5117 - loss: 3.5460 - recall: 0.5416 - val_accuracy: 0.4878 - val_loss: 0.6939 - val_recall: 0.6371
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5094 - loss: 0.7982 - recall: 0.4937 - val_accuracy: 0.5061 - val_loss: 0.6942 - val_recall: 1.0000
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5234 - loss: 0.6978 - recall: 0.5157 - val_accuracy: 0.5653 - val_loss: 0.6887 - val_recall: 0.5081
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5235 - loss: 0.6947 - recall: 0.6030 - val_accuracy: 0.5347 - val_loss: 0.6926 - val_recall: 0.1734
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4997 - loss: 0.6962 - recall: 0.4038 - val_accuracy: 0.5184 - val_loss: 0.6924 - val_recall: 0.7823
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5277 - loss: 0.6940 - recall: 0.6339 - val_accuracy: 0.5204 - val_loss: 0.6909 - val_recall: 0.4032
Epoch 7/50
31/31 ━━━━━━━━━━

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_33 (Conv1D)              │ (None, 40, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_34 (Conv1D)              │ (None, 38, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_21 (MaxPooling1D) │ (None, 19, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 19, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 1216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 128)            │       155,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 486,725 (1.86 MB)

 Trainable params: 162,241 (633.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 324,484 (1.24 MB)

In [38]:
# Reshape X_test for evaluation
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Evaluate the model
evaluation_results = model.evaluate(X_test_reshaped, y_test)

# Extracting individual evaluation metrics
loss = evaluation_results[0]
accuracy = evaluation_results[1]

# Printing evaluation results
print("Evaluation Results:")
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)




20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 950us/step - accuracy: 0.5913 - loss: 0.6610 - recall: 0.7298
Evaluation Results:
Test Loss: 0.664807915687561
Test Accuracy: 0.5784313678741455


## Model 3


In [54]:
from keras.layers import Dropout, AveragePooling1D, MaxPooling1D, Flatten, Dense

# Define the model
model = Sequential()

# Add convolutional layers

# Layer 1: Conv1D
model.add(Conv1D(filters=512, kernel_size=7, strides=1, padding='same', activation='relu', input_shape=(X_train.shape[1], 1)))
# Layer 2: AveragePooling1D
model.add(AveragePooling1D(pool_size=5, strides=2, padding='same'))
# 3 Conv1D
model.add(Conv1D(filters=256, kernel_size=5, strides=2, padding='same',activation='relu'))
# 4 maxpooling
model.add(MaxPooling1D(pool_size=6, strides=2, padding='same'))
# 5 conv1d
model.add(Conv1D(filters=128, kernel_size=5, strides=2, padding='same', activation='relu'))
# 6 maxpooling1d
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

# Flatten the output
model.add(Flatten())


# Fully Connected Layer 1
model.add(Dense(128, activation='relu'))
# Fully Connected Layer 2
model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.0001)  # Adjusted learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'recall'])


X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
# Train the model
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_split=0.2)

# model.summary()


# Reshape X_test for evaluation
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Evaluate the model
evaluation_results = model.evaluate(X_test_reshaped, y_test)

# Extracting individual evaluation metrics
loss = evaluation_results[0]
accuracy = evaluation_results[1]

# Printing evaluation results
print("Evaluation Results:")
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4917 - loss: 0.7867 - recall: 0.5383 - val_accuracy: 0.4939 - val_loss: 0.6921 - val_recall: 0.9476
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5161 - loss: 0.6991 - recall: 0.7438 - val_accuracy: 0.5204 - val_loss: 0.6908 - val_recall: 0.1129
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4973 - loss: 0.7066 - recall: 0.6337 - val_accuracy: 0.5061 - val_loss: 0.7042 - val_recall: 1.0000
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5288 - loss: 0.6909 - recall: 0.7332 - val_accuracy: 0.5510 - val_loss: 0.6877 - val_recall: 0.2702
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4934 - loss: 0.7036 - recall: 0.5812 - val_accuracy: 0.5531 - val_loss: 0.6866 - val_recall: 0.5202
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5276 - loss: 0.6928 - recall: 0.6213 - val_accuracy: 0.5061 - val_loss: 0.6994 - val_recall: 1.0000
Epoch 7/50

In [58]:
from keras.layers import Dropout, AveragePooling1D, MaxPooling1D, Flatten, Dense

# Define the model
model = Sequential()

# Add convolutional layers

# Layer 1: Conv1D
model.add(Conv1D(filters=512, kernel_size=7, strides=1, padding='same', activation='relu', input_shape=(X_train.shape[1], 1)))
# Layer 2: AveragePooling1D
model.add(AveragePooling1D(pool_size=5, strides=2, padding='same'))
# 3 Conv1D
model.add(Conv1D(filters=256, kernel_size=5, strides=2, padding='same',activation='relu'))
# 4 maxpooling
model.add(MaxPooling1D(pool_size=6, strides=2, padding='same'))
# 5 conv1d
model.add(Conv1D(filters=128, kernel_size=5, strides=2, padding='same', activation='relu'))
# 6 maxpooling1d
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

# Flatten the output
model.add(Flatten())


# Fully Connected Layer 1
model.add(Dense(128, activation='relu'))
# Fully Connected Layer 2
model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.000001)  # Adjusted learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'recall'])


X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
# Train the model
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_split=0.2)

# model.summary()


# Reshape X_test for evaluation
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Evaluate the model
evaluation_results = model.evaluate(X_test_reshaped, y_test)

# Extracting individual evaluation metrics
loss = evaluation_results[0]
accuracy = evaluation_results[1]

# Printing evaluation results
print("Evaluation Results:")
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4945 - loss: 0.8458 - recall: 0.5462 - val_accuracy: 0.4878 - val_loss: 0.6925 - val_recall: 0.1210
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4903 - loss: 0.7926 - recall: 0.4292 - val_accuracy: 0.4898 - val_loss: 0.6933 - val_recall: 0.0282
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5006 - loss: 0.7536 - recall: 0.4450 - val_accuracy: 0.4857 - val_loss: 0.6930 - val_recall: 0.0242
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5076 - loss: 0.7312 - recall: 0.3673 - val_accuracy: 0.5429 - val_loss: 0.6915 - val_recall: 0.5242
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4820 - loss: 0.7289 - recall: 0.4918 - val_accuracy: 0.5184 - val_loss: 0.6914 - val_recall: 0.2056
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5163 - loss: 0.7089 - recall: 0.4418 - val_accuracy: 0.5163 - val_loss: 0.6909 - val_recall: 0.2258
Epoch 7/50